#### There is a sequence in the images provided and I thought it will be interesting to visualize annotations of each case per day by viewing the slices provided as a short video which one can control. It will also help to analyse the quality of prediction on validation data once rendered correctly. 

# **Import Packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import animation
from matplotlib.patches import Rectangle
import os
import glob
from IPython import display
import matplotlib.widgets
import cv2
import warnings
warnings.filterwarnings('ignore')

# **Import Training Data**

In [ ]:
root = "../input/uw-madison-gi-tract-image-segmentation/"
train = pd.read_csv(root + "train.csv").\
           replace({"large_bowel":"lb","small_bowel":"sb","stomach":"st"}).\
           set_index(['id','class']).unstack()['segmentation'].\
           reset_index().\
           rename_axis(None, axis=1)

# **Process Training Data**

In [ ]:
# Process Image MetaData

image_files = glob.glob(root + "train/**/*.png", recursive=True)
image_files_df = pd.DataFrame({"path":image_files})
image_files_df["files_cleaned"] = image_files_df["path"].map(lambda x: x.lstrip("../input/uw-madison-gi-tract-image-segmentation"))
image_files_df[['case','case_day','scans','slice']] = image_files_df['files_cleaned'].str.split('/', -1, expand=True)
image_files_df = image_files_df.drop(["files_cleaned","scans"],axis = 1)
image_files_df[["slice","slice_id","image_meta"]] = image_files_df['slice'].str.split('_', 2, expand=True)
image_files_df['id'] = image_files_df["case_day"]+ "_" + image_files_df["slice"] + "_" + image_files_df["slice_id"]
image_files_df["image_meta"] = image_files_df["image_meta"].map(lambda x: x.rstrip(".png"))
image_files_df[['height','width','ht_pxl','wd_pxl']] = image_files_df["image_meta"].str.split('_', -1, expand=True)
image_files_df['height'] = image_files_df['height'].astype(int)
image_files_df['width'] = image_files_df['width'].astype(int)
image_files_df['ht_pxl'] = image_files_df['ht_pxl'].astype(float)
image_files_df['wd_pxl'] = image_files_df['wd_pxl'].astype(float)
image_files_df = image_files_df[['path','id','height','width','ht_pxl','wd_pxl']].drop_duplicates()

# Process Train Data

train_processed = train.copy()
train_processed[['case','day','slice_name','slice']] = train_processed['id'].str.split('_', -1, expand=True)
train_processed = train_processed.drop(["slice_name"], axis = 1)
train_processed["case"] = train_processed["case"].map(lambda x: x.lstrip('case'))
train_processed["day"] = train_processed["day"].map(lambda x: x.lstrip('day'))

# Final Train Data
train_final = pd.merge(train_processed,image_files_df,how = "left", on = ["id"])

# **Helper Functions**

Excellent helper functions were provided in this [notebook](https://www.kaggle.com/code/dschettler8845/uwm-gi-tract-image-segmentation-eda) which I have reused here

In [ ]:
def case_function(case_id,data):
    return data[data["id"] == case_id].squeeze()

def rle_decode(mask_rle, shape, color=1):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns: 
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    # Split the string by space, then convert it into a integer array
    s = np.array(mask_rle.split(), dtype=int)

    # Every even value is the start, every odd value is the "run" length
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths

    # The image image is actually flattened since RLE is a 1D "run"
    if len(shape)==3:
        h, w, d = shape
        img = np.zeros((h * w, d), dtype=np.float32)
    else:
        h, w = shape
        img = np.zeros((h * w,), dtype=np.float32)

    # The color here is actually just any integer you want!
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
        
    # Don't forget to change the image back to the original shape
    return img.reshape(shape)

def open_gray16(_path, normalize=True, to_rgb=False):
    """ Helper to open files """
    if normalize:
        if to_rgb:
            return np.tile(np.expand_dims(cv2.imread(_path, cv2.IMREAD_ANYDEPTH)/65535., axis=-1), 3)
        else:
            return cv2.imread(_path, cv2.IMREAD_ANYDEPTH)/65535.
    else:
        if to_rgb:
            return np.tile(np.expand_dims(cv2.imread(_path, cv2.IMREAD_ANYDEPTH), axis=-1), 3)
        else:
            return cv2.imread(_path, cv2.IMREAD_ANYDEPTH)

def overlay_plot(case_data,data):
    img = open_gray16(case_data["path"], to_rgb=True)
    img = ((img-img.min())/(img.max()-img.min())).astype(np.float32)
    seg_rgb = np.stack(
        [rle_decode(case_data[f"{_seg_type}"], 
                shape=(case_data["width"], case_data["height"]), color=1) if not pd.isna(case_data[f"{_seg_type}"]) else np.zeros((case_data["width"], case_data["height"])) for _seg_type in ["lb", "sb", "st"]], axis=-1).astype(np.float32)
    seg_overlay = cv2.addWeighted(src1=img, alpha=0.99, 
                              src2=seg_rgb, beta=0.33, gamma=0.0)
    return seg_overlay

# **Image GIF Generator**

In [ ]:
def scan_as_gif(data,case_id,day,fps):
    rel_cases = data[data["id"].str.contains(f"case{case_id}_day{day}")]["id"].tolist()

    frames = []
    for i in rel_cases:
        case_data = case_function(i,train_final)
        seg_overlay = overlay_plot(case_data,train_final)
        frames.append(seg_overlay)

    plt.figure(figsize=(8,8))
    patch = plt.imshow(frames[0]) 
    plt.title(f"Segmentation Overlay For ID: {case_function(rel_cases[0],train_final)['id']}", fontweight="bold")
    handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
    labels = ["Large Bowel Segment", "Small Bowel Segmentat", "Stomach Segment"]
    plt.legend(handles,labels)
    plt.axis(False)

    def animate(i):
        plt.title(f"Segmentation Overlay For ID: {case_function(rel_cases[i],train_final)['id']}", fontweight="bold")
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), 
                                       animate, 
                                       frames = len(frames), 
                                       interval=15)
    video = anim.to_jshtml(fps = fps)
    html = display.HTML(video)
    return display.display(html)

# **Rendering**

In [ ]:
scan_as_gif(data = train_final,
            case_id = 101,
            day = 20,
            fps = 8
           )
plt.close()